# 行业 · CFD 数据管理
本 Notebook 用于管理 CFD 经纪商（如 TMGM）及其新闻：新增 / 编辑 / 删除。

如未安装交互组件：在下方单元运行 `!pip install ipywidgets` 并重启内核。


In [1]:
# 基础导入
from ipywidgets import VBox, HBox, Button, Text, Dropdown, Label, HTML, Layout, Output
from cfd_admin import CFDAdmin

admin = CFDAdmin()
out = Output()
out2 = Output()
display(HTML('<b>数据库:</b> shopback_data.db'))


HTML(value='<b>数据库:</b> shopback_data.db')

In [5]:
# —— 经纪商管理 ——
def load_brokers():
    brokers = admin.list_brokers()
    options = [(f"{b.name} (id={b.id})", b.id) for b in brokers]
    return brokers, options

brokers, broker_options = load_brokers()
broker_select = Dropdown(options=broker_options, description='选择: ', layout=Layout(width='400px'))
name_txt = Text(description='名称:', placeholder='如 TMGM', layout=Layout(width='400px'))
regs_txt = Text(description='监管:', placeholder='如 ASIC, VFSCN', layout=Layout(width='400px'))
rating_txt = Text(description='评分:', placeholder='如 A+', layout=Layout(width='200px'))
site_txt = Text(description='官网:', placeholder='如 https://tmgm.com', layout=Layout(width='400px'))
logo_txt = Text(description='Logo:', placeholder='/static/uploads/cfd/xxx.png', layout=Layout(width='400px'))

btn_create = Button(description='新增', button_style='success')
btn_update = Button(description='保存修改', button_style='info')
btn_delete = Button(description='删除', button_style='danger')

def fill_form(broker_id):
    if not broker_id:
        name_txt.value = ''
        regs_txt.value = ''
        rating_txt.value = ''
        site_txt.value = ''
        logo_txt.value = ''
        return
    b = admin.get_broker(int(broker_id))
    name_txt.value = b.name or ''
    regs_txt.value = b.regulators or ''
    rating_txt.value = b.rating or ''
    site_txt.value = b.website or ''
    logo_txt.value = b.logo_url or ''

def refresh_brokers(select_latest=False):
    global brokers, broker_options
    brokers, broker_options = load_brokers()
    if not brokers:
        broker_select.options = []
        fill_form(None)
    else:
        broker_select.options = broker_options
        if select_latest:
            broker_select.value = broker_options[0][1]
        fill_form(broker_select.value if broker_select.options else None)

def on_select_change(change):
    if change['name'] == 'value':
        fill_form(change['new'])

broker_select.observe(on_select_change, names='value')

@out.capture(clear_output=True)
def do_create(_):
    b = admin.create_broker(name_txt.value.strip(), regs_txt.value.strip() or None, rating_txt.value.strip() or None, site_txt.value.strip() or None, logo_txt.value.strip() or None)
    print('已新增经纪商:', b)
    refresh_brokers(select_latest=True)

@out.capture(clear_output=True)
def do_update(_):
    if not broker_select.options:
        print('无可选经纪商')
        return
    b = admin.update_broker(int(broker_select.value), name=name_txt.value.strip() or None, regulators=regs_txt.value.strip() or None, rating=rating_txt.value.strip() or None, website=site_txt.value.strip() or None, logo_url=logo_txt.value.strip() or None)
    print('已保存:', b)
    refresh_brokers()

@out.capture(clear_output=True)
def do_delete(_):
    if not broker_select.options:
        print('无可选经纪商')
        return
    admin.delete_broker(int(broker_select.value), cascade=True)
    print('已删除')
    refresh_brokers()

btn_create.on_click(do_create)
btn_update.on_click(do_update)
btn_delete.on_click(do_delete)

display(HTML('<h3>经纪商管理</h3>'))
display(HBox([broker_select]))
display(VBox([name_txt, regs_txt, HBox([rating_txt, site_txt]), logo_txt, HBox([btn_create, btn_update, btn_delete]), out]))

refresh_brokers()


HTML(value='<h3>经纪商管理</h3>')

In [ ]:
# —— 新闻管理 ——
def broker_pairs():
    bs = admin.list_brokers()
    return [(f"{b.name} (id={b.id})", b.id) for b in bs]

broker_for_news = Dropdown(options=broker_pairs(), description='经纪商:', layout=Layout(width='400px'))
news_select = Dropdown(options=[], description='新闻:', layout=Layout(width='400px'))
news_title = Text(description='标题:', layout=Layout(width='500px'))
news_tag = Text(description='标签:', placeholder='如 行业观察/简讯', layout=Layout(width='300px'))
btn_news_create = Button(description='新增新闻', button_style='success')
btn_news_update = Button(description='保存修改', button_style='info')
btn_news_delete = Button(description='删除', button_style='danger')

def refresh_news():
    news = []
    if broker_for_news.value is not None:
        news = admin.list_news(int(broker_for_news.value))
    news_options = [(f"{n.title} (id={n.id})", n.id) for n in news]
    news_select.options = news_options
    if news_options:
        news_select.value = news_options[0][1]
        # 填充表单
        n = admin.list_news(int(broker_for_news.value))[0]
        news_title.value = n.title
        news_tag.value = n.tag or ''
    else:
        news_title.value = ''
        news_tag.value = ''

def on_broker_change(change):
    if change['name'] == 'value':
        refresh_news()

def on_news_change(change):
    if change['name'] == 'value' and change['new'] is not None:
        nid = int(change['new'])
        # 读取并填入
        ns = [n for n in admin.list_news(int(broker_for_news.value)) if n.id == nid]
        if ns:
            news_title.value = ns[0].title
            news_tag.value = ns[0].tag or ''

@out2.capture(clear_output=True)
def do_news_create(_):
    if broker_for_news.value is None:
        print('请先选择经纪商')
        return
    n = admin.create_news(int(broker_for_news.value), news_title.value.strip(), news_tag.value.strip() or None)
    print('已新增新闻:', n)
    refresh_news()

@out2.capture(clear_output=True)
def do_news_update(_):
    if not news_select.options:
        print('无可选新闻')
        return
    n = admin.update_news(int(news_select.value), title=news_title.value.strip() or None, tag=news_tag.value.strip() or None)
    print('已保存:', n)
    refresh_news()

@out2.capture(clear_output=True)
def do_news_delete(_):
    if not news_select.options:
        print('无可选新闻')
        return
    admin.delete_news(int(news_select.value))
    print('已删除')
    refresh_news()

broker_for_news.observe(on_broker_change, names='value')
news_select.observe(on_news_change, names='value')
btn_news_create.on_click(do_news_create)
btn_news_update.on_click(do_news_update)
btn_news_delete.on_click(do_news_delete)

display(HTML('<h3>新闻管理</h3>'))
display(HBox([broker_for_news]))
display(HBox([news_select]))
display(VBox([news_title, news_tag, HBox([btn_news_create, btn_news_update, btn_news_delete]), out2]))

# 初始化
broker_for_news.options = broker_pairs()
refresh_news()


HTML(value='<h3>新闻管理</h3>')